In [2]:
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

test = pd.read_csv("./data/X_test.csv")
train = pd.read_csv("./data/X_train.csv") 


In [345]:
def predict_by_svd(df, targetUsers, targetProduct, usersCompare, usersProd):
    df_Score = df.pivot(
        index = 'UserId',
        columns = 'ProductId',
        values = 'Score'
    ).fillna(0)

    from numpy.linalg import svd

    userMean = np.mean(df_Score, axis = 1).to_numpy()
    # df['user_mean_score'].fillna(mean_score, inplace = True)
    matrix_mean = df_Score - userMean.reshape(-1, 1)

    U, sigma, vt = svd(matrix_mean, full_matrices = False)
        
    sigma = np.diag(sigma)

    MatrixApprox = np.dot(np.dot(U, sigma), vt) + userMean.reshape(-1,1)
    Matrix = pd.DataFrame(MatrixApprox, index =  usersCompare, columns= usersProd)
    index = Matrix.index.values.tolist()
    if (targetUsers in index) == False:
        return
    else:
        result = Matrix.loc[targetUsers][targetProduct]
        return result.round(1) 

In [ ]:
train.index.values.tolist()

In [346]:
testAct = test[['ProductId', 'UserId', 'Score']]
testProd = list(testAct['ProductId'])
testUser = list(testAct['UserId'])
len(testProd)

300000

In [ ]:
def predict_by_comparing_users(df):
    pred_score = []
    testAct = df[['ProductId', 'UserId', 'Score']]
    testProd = list(testAct['ProductId'])
    testUser = list(testAct['UserId'])

    for i in range(len(testUser)): 
        if (i+1)%100 == 0:
            print('Review {} of {}'.format(i+1, len(testUser)))
        #Target UserId:
        targetUsers = testUser[i]
        #Target Product:
        targetProduct = testProd[i]
    
        #Listof users watched Target Product:
        productScored = train[train['ProductId']==targetProduct]
        sameUser = list(productScored['UserId'].values)
        sameUser += [targetUsers]

        #Dataframe of product that target user watched along with other users:
        usersProd = list(train[train['UserId']==targetUsers].ProductId)
        usersProd += [targetProduct]
        usersSameProd = train[train['ProductId'].isin(usersProd)]  
        
        #Final DF that we are going to looking for 
        finalDf= usersSameProd[usersSameProd['UserId'].isin(sameUser)]
        finalDf.groupby('UserId').Score.count()
        usersCompare = list(finalDf.groupby('UserId').groups)
 
        pred_score += [predict_by_svd(finalDf, targetUsers, targetProduct, usersCompare, usersProd)]

    return pred_score
    
prediction = predict_by_comparing_users(test)
    

In [ ]:
testUserId = pd.DataFrame(test['UserId'])
testUserId['Predict_comparing_users'] = prediction
testUserId
testUserId.to_csv("predict_test.csv", index= None)
testUserId[6000:10000]

,UserId,Predict_comparing_users
6000,A3L5NHE84NCRCX,5.0
6001,A1865MB0KN26D6,4.0
6002,A1A4OI8NWWCZDZ,5.0
6003,A1RV9CQTA666FH,5.0
6004,A2BSZPRBKYAB2K,5.0
...,...,...
9995,A3EANXZM97V8YS,5.0
9996,A2J3MR64L567OK,1.0
9997,A39CX0EE4BZCZC,5.0
9998,A31O7B59DW7OHR,5.0
